In [22]:
import pandas as pd
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import numpy as np
from tqdm import tqdm

In [23]:
path="C:/Quant/project/"
targetpath ="TRAIN.csv"
valpath = "val.csv"
testpath ="test.csv"

In [24]:
df= pd.read_csv(path+valpath)

In [25]:
df

,Unnamed: 0,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0,22_58,22,58,0.600911,1.081210,-1.635753,0.082000,-0.654596,2.116271,...,1.829991,1.225630,-0.133157,0.338368,-1.329777,0.458489,-1.171452,-1.116468,-1.403561,1.496008
1,1,22_59,22,59,-0.148273,-1.107688,-0.762889,0.759441,-0.661708,0.614619,...,0.329355,1.225630,0.335490,0.338368,0.752005,0.366660,-1.171452,0.843250,0.520654,0.824550
2,2,22_61,22,61,-0.595294,0.776524,0.546407,0.045971,-0.606577,-0.313791,...,-1.076004,-0.815907,-0.349123,0.338368,0.752005,-0.404179,0.084959,-0.250347,-0.819772,-0.118027
3,3,22_62,22,62,-0.983207,-0.227873,0.400930,-1.453166,0.601058,4.339244,...,0.840153,-0.815907,-0.667887,0.338368,-1.329777,3.346428,-1.171452,-0.388681,1.211371,4.564400
4,4,22_63,22,63,-0.229776,-0.004463,-0.035502,2.124675,-0.658155,-0.573081,...,0.808395,-0.815907,-0.425994,0.338368,0.752005,-0.606159,-1.171452,0.717530,-1.274164,-0.703374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,24995,33_213,33,213,-1.414616,0.833006,-1.078762,-0.972510,2.457652,-0.235570,...,-0.120991,-0.657001,-0.324576,0.173576,0.825183,-0.382438,-0.283733,-0.444719,1.520904,-0.176580
24996,24996,33_214,33,214,-0.006400,-0.584242,0.633364,0.188913,-0.773373,-0.483320,...,0.446051,-0.657001,-0.452254,0.173576,0.825183,-0.401591,-0.283733,-0.329566,0.846726,-0.484327
24997,24997,33_216,33,216,-0.275792,0.087031,1.204072,0.267470,-0.389980,-0.500376,...,-0.619648,-0.657001,-1.341064,0.173576,-1.211851,-0.528420,0.992241,-0.913289,-0.411646,-0.506684
24998,24998,33_217,33,217,-0.934072,0.528163,-1.649470,-1.010666,1.146295,-0.087223,...,0.394762,-0.657001,-0.934546,0.173576,0.825183,-0.255277,0.992241,-0.845721,-0.133167,-0.050718


In [26]:
#两方面去考虑attention,首先,可以忽视事件数据,将所有Id一样的人聚集在一起,将他们视作一个sequence,将他们的target作为另一个,观察两个sequence的关系
#其次,可以以同一个时域做Attention,将这个时域中的数据作为一个切面sequence,然后跟他们的结果做相关性

In [27]:
def GetDataFromKaggle():
    print("successful get date")

In [28]:
def GetMaxFeature(x):
    id_size = x["investment_id"].nunique()
    time_id_size = x["time_id"].nunique()
    return id_size,time_id_size

In [29]:
class Ubiquant(Dataset):
    def __init__(self,datapath,context = 100,train_data = True,get_time_data = True):
        super(Ubiquant,self).__init__()
        self.get_time = get_time_data
        if(train_data):
            self.data = pd.read_csv(datapath)
            self.id_size,self.time_size = GetMaxFeature(self.data)
            if(get_time_data):#get an x,y base on time frame
                self.y = self.data.loc[:,"target"]
                self.x = self.data.drop(columns=["Unnamed: 0","row_id","target"])
                self.x.loc[:,"time_id"] -=self.x.loc[0,"time_id"]
                #self.id = self.x[["time_id"]]
                self.id = self.x[["investment_id"]]
                self.x = self.x.drop(columns=["investment_id"])
                #self.x = self.x.drop(columns=["time_id"])
            else:#get (x,y) base on investment_id
                self.data = self.data.sort_values(by=["investment_id"])
                self.y = self.data.loc[:,"target"]
                self.x = self.data.drop(columns=["Unnamed: 0","row_id","time_id","target"])
                self.id = self.x[["investment_id"]]
                self.x = self.x.drop(columns=["investment_id"])
        
        #pad context before the data
        self.context = context
        self.pad_id =np.pad(self.id.to_numpy(),((context-1,0),(0,0)),constant_values = 0)
        self.pad_x =np.pad(self.x.to_numpy(),((context-1,0),(0,0)),constant_values = 0)
        self.y =self.y.to_numpy()
        
        
        self.pad_id =torch.LongTensor(self.pad_id)
        self.pad_x = torch.FloatTensor(self.pad_x)
        self.y = torch.FloatTensor(self.y)
        
        print(f"succesfully load data with shape x : {self.x.shape} | shape y :{self.y.shape} | get time data is: {get_time_data}")
        
                
            
    def __getitem__(self,index):
        context = self.context
        return self.pad_id[index:index+context],self.pad_x[index:index+context],self.y[index]
        
    def __len__(self):
        return self.x.shape[0]
    
    def GetSize(self):
        if(get_time_data):
            return self.time_size
        else:
            return self.id_size

            
            

In [30]:
def LoadData(get_time_data = True):
    trainSet =Ubiquant(path+targetpath,context = config["context"], get_time_data = get_time_data)
    valSet =Ubiquant(path+valpath, context = config["context"], get_time_data = get_time_data)
    testSet =Ubiquant(path+testpath, context = config["context"], get_time_data = get_time_data)
    trainLoader = DataLoader(trainSet,batch_size = config["batch_size"],shuffle =True,drop_last=True)#,num_workers=2)
    valLoader = DataLoader(valSet,batch_size = config["batch_size"],shuffle =True,drop_last=True)#,num_workers=2)
    testLoader = DataLoader(testSet,batch_size = config["batch_size"],shuffle =False,drop_last=False)
    return trainLoader,valLoader,testLoader

In [31]:
class SimpleAttention(nn.Module):
    def __init__(self,seq_len = 32, input_feature =311):
        super(SimpleAttention,self).__init__() 
        
        self.id_embedding = nn.Embedding(10000,11)
        self.feature_embedding = nn.Sequential(
            nn.Conv1d(seq_len,64,3,padding ="same"),
            nn.BatchNorm1d(64),
            nn.Sigmoid(),
            nn.Conv1d(64,128,3,padding = "same"),
            nn.BatchNorm1d(128),
            nn.Sigmoid(),
            nn.Conv1d(128,seq_len,3,padding ="same"),
            
        )
        self.att = nn.MultiheadAttention(input_feature,8,batch_first=True)
        self.flat = nn.Flatten()
        self.MLP =nn.Sequential(
            nn.Linear(input_feature*seq_len,64),
            nn.BatchNorm1d(64),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(128,256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(256,512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(128,8),
            nn.BatchNorm1d(8),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(8,1)
        )
            
        self.criterion = nn.MSELoss()
        
    def forward(self,_id,f_features):
        invest_embedding = self.id_embedding(_id).squeeze(dim=2)
        f_features = self.feature_embedding(f_features)
        #print(invest_embedding.shape)
        #print(f_features.shape)
        _input =torch.cat((invest_embedding,f_features),axis =-1)
        #print(_input.shape)
        #_input = _input.unsqueeze(dim=1)
        #print(_input.shape)
        output,_ = self.att(_input,_input,_input)
        #print(output.shape)
        output = self.flat(output)
        #print(output.shape)
        output = self.MLP(output)
        return output
    
    def cal_Loss(self,y_hat,y):
        return self.criterion(y_hat,y)

In [32]:
class SimpleMLP(nn.Module):
    def __init__(self,seq_len = 32,input_feature = 311):
        super(SimpleMLP,self).__init__()
        
        self.id_embedding = nn.Embedding(10000,11)
        self.flat = nn.Flatten()
        self.MLP =nn.Sequential(
            nn.Linear(input_feature*seq_len,64),
            nn.BatchNorm1d(64),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(64,128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(128,256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(256,512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(128,8),
            nn.BatchNorm1d(8),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(8,1)
        )
        
        self.criterion = nn.MSELoss()
        
    def forward(self,_id,f_features):
        invest_embedding = self.id_embedding(_id).squeeze(dim=2)
     
        #print(invest_embedding.shape)
        #print(f_features.shape)
        _input =torch.cat((invest_embedding,f_features),axis =-1)
        #print(_input.shape)
        #_input = _input.unsqueeze(dim=1)
        #print(_input.shape)
        _input = self.flat(_input)
        output = self.MLP(_input)
        return output

    def cal_Loss(self,y_hat,y):
        return self.criterion(y_hat,y)

In [33]:
def get_corr(y,target):
    y,target = y.reshape(-1),target.reshape(-1)
    ymean,targetmean = torch.mean(y),torch.mean(target)
    
    vy = y-ymean
    vt = target-targetmean
    
    corr = torch.sum(vy*vt)/(torch.sqrt(torch.sum(vy**2))* torch.sqrt(torch.sum(vt**2)))
    return corr

In [34]:
def train(model,trainLoader,valLoader, optimizer, scheduler, config):
    best_loss =1000
    epochs =config["epoch"]
    for epoch in range(epochs):
        batch_bar = tqdm(total = len(trainLoader),dynamic_ncols = True,leave =False,position = 0,desc = "train")
        model.train()
        train_total_loss = 0
        for i,(x1,x2,y) in enumerate(trainLoader):
            optimizer.zero_grad()
            x1,x2,y = x1.cuda(),x2.cuda(),y.cuda()
            y_hat = model(x1,x2)
            y_hat = y_hat.reshape(-1,1)
            y =y.reshape(-1,1)
            loss = model.cal_Loss(y_hat,y)
            train_total_loss+=float(loss.cpu())
            batch_bar.set_postfix(
                loss="{:.04f}".format(float(train_total_loss / (i + 1))),
                lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
            loss.backward()
            optimizer.step()
            scheduler.step()
            batch_bar.update()
        batch_bar.close()
        
        val_total_loss = 0
        val_total_corr = 0
        batch_bar = tqdm(total = len(valLoader),dynamic_ncols = True,leave =False,position = 0,desc = "val")
        model.eval()
        for i,(x1,x2,y) in enumerate(valLoader):
            with torch.no_grad():
                x1,x2,y = x1.cuda(),x2.cuda(),y.cuda()
                y_hat = model(x1,x2)
                y_hat = y_hat.reshape(-1,1)
                y =y.reshape(-1,1)
                loss = model.cal_Loss(y_hat,y)
                

                corr = get_corr(y_hat,y)
                val_total_corr+=corr
               
                val_total_loss+=loss
            batch_bar.set_postfix(
                loss="{:.04f}".format(float(val_total_loss / (i + 1))),
                
                corr="{:.04f}".format(float(val_total_corr / (i + 1))),
             )
            batch_bar.update()
        batch_bar.close()
        val_loss = float(val_total_loss/len(valLoader))
        if(val_loss<best_loss):
            best_loss = val_loss
            torch.save(model.state_dict(),config["store_path"])
            print("successfully save model")
        
        print(f"Epoch {epoch+1}/{epochs}: train loss {float(train_total_loss / len(trainLoader)):0.04f}, Learning Rate {optimizer.param_groups[0]['lr']:0.04f}, val loss{float(val_total_loss/len(valLoader)):0.04f}, corr {float(val_total_corr / (i + 1)):0.04f}")

In [35]:
def test(model,testLoader,config):
    model.load_state_dict(torch.load(config["store_path"]))
    model.eval()
    test_total_loss = 0
    test_total_corr = 0
    batch_bar = tqdm(total = len(testLoader),dynamic_ncols = True,leave =False,position = 0,desc = "Test")
    for i,(x1,x2,y) in enumerate(testLoader):
        with torch.no_grad():
            x1,x2,y = x1.cuda(),x2.cuda(),y.cuda()
            y_hat = model(x1,x2)
            y_hat = y_hat.reshape(-1,1)
            y =y.reshape(-1,1)
            loss = model.cal_Loss(y_hat,y)
                

            corr = get_corr(y_hat,y)
            test_total_corr+=corr
            test_total_loss+=loss
        batch_bar.set_postfix(
                test_loss="{:.04f}".format(float(test_total_loss / (i + 1))),
                
                test_corr="{:.04f}".format(float(test_total_corr / (i + 1))),
             )
        batch_bar.update()
    batch_bar.close()
    print(f"test loss{float(test_total_loss/len(testLoader)):0.04f},test corr {float(test_total_corr / len(testLoader)):0.04f}")

In [36]:
def Main():
    #set all parameter
    trainLoader,valLoader,testLoader = LoadData(get_time_data = True)
    model = SimpleAttention(config["context"],312).cuda()
    model.train()
    baseline_model = SimpleMLP(config["context"],312).cuda()
    baseline_model.train()
    optimizer = getattr(torch.optim,config["optimz"])(model.parameters(),lr =config["learning_rate"])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(trainLoader) * config["epoch"]))
    
    
    base_optimizer = getattr(torch.optim,config["optimz"])(model.parameters(),lr =config["learning_rate"])
    base_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(trainLoader) * config["epoch"]))
    #params
    num_para =0
    for p in model.parameters():
        num_para+=p.numel()
    print(f'Number of params : {num_para}')
    
    train(model,trainLoader,valLoader,optimizer,scheduler,config)
    print("///////////Start Train Base Line//////////")
    train(baseline_model,trainLoader,valLoader,base_optimizer,base_scheduler,base_config)
    
    print("//////////Start test///////////")
    print("----BaseLine-----")
    test(baseline_model,testLoader,base_config)
    print("----Attention-----")
    test(model,testLoader,config)

In [41]:
config={"epoch" : 100,"batch_size":64,"learning_rate":0.1,"optimz":"Adam","store_path":"C:/Quant/project/Att.pth","context":1}

In [42]:
base_config={"epoch" :100,"batch_size":64,"learning_rate":0.1,"optimz":"Adam","store_path":"C:/Quant/project/MLP.pth","context":1}

In [43]:
Main()

succesfully load data with shape x : (50000, 301) | shape y :torch.Size([50000]) | get time data is: True
succesfully load data with shape x : (25000, 301) | shape y :torch.Size([25000]) | get time data is: True
succesfully load data with shape x : (25000, 301) | shape y :torch.Size([25000]) | get time data is: True
Number of params : 887282


successfully save model
Epoch 1/100: train loss 0.8985, Learning Rate 0.1000, val loss0.8941, corr -0.0010


successfully save model
Epoch 2/100: train loss 0.8958, Learning Rate 0.0999, val loss0.8676, corr -0.0770


successfully save model
Epoch 3/100: train loss 0.8917, Learning Rate 0.0998, val loss0.8628, corr 0.0826


Epoch 4/100: train loss 0.8911, Learning Rate 0.0996, val loss0.8755, corr 0.0151


...

Epoch 98/100: train loss 1.0866, Learning Rate 0.1000, val loss0.8734, corr -0.0351


Epoch 99/100: train loss 1.0806, Learning Rate 0.1000, val loss0.8718, corr -0.0355


Epoch 100/100: train loss 1.0774, Learning Rate 0.1000, val loss0.8718, corr -0.0331
//////////Start test///////////
----BaseLine-----


test loss0.8627,test corr -0.0148
----Attention-----


test loss0.8542,test corr 0.0951
